### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

### Loading Data

In [2]:
with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/train_set/x_train.pickle", "rb") as file:
    x_train = pickle.load(file)

with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/train_set/y_train.pickle", "rb") as file:
    y_train = pickle.load(file)

with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/internal_test_set/internal_x_test.pickle", "rb") as file:
    internal_x_test = pickle.load(file)

with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/internal_test_set/internal_y_test.pickle", "rb") as file:
    internal_y_test = pickle.load(file)

with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/external_test_set/external_x_test.pickle", "rb") as file:
    external_x_test = pickle.load(file)

with open("../../../../data/2_all_active_autoencoder_inactive/2DRDKit/for_ml/external_test_set/external_y_test.pickle", "rb") as file:
    external_y_test = pickle.load(file)

In [3]:
pd.options.display.max_columns = 300

In [4]:
x_train_df = pd.DataFrame(data=x_train)
x_train_df[x_train_df[13].isnull()]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
332,12.138731,0.078403,12.138731,0.078403,0.622431,274.181,265.109,274.984935,78.0,0.0,NaN,NaN,NaN,NaN,0.875,1.5625,2.3125,2.339042,681.724018,10.957819,8.459863,10.172935,7.860173,5.02607,6.648715,3.522526,5.594479,2.482263,4.815131,1.70548,3.799793,-1.570519,9923.302413,9.640986,3.779548,1.452937,98.738738,0.0,0.0,0.0,0.0,0.0,98.579986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.379009,0.0,3.562482,0.0,0.0,0.0,64.951108,0.0,5.687386,5.559267,0.0,0.0,0.0,18.295119,0.0,0.0,0.0,59.391841,0.0,15.333759,0.0,22.0,0.0,4.794537,0.0,20.291904,0.0,11.07361,4.260148,0.0,58.159786,0.0,0.0,3.04292,0.078403,12.138731,0.853105,1.116204,0.0,17.687305,0.0,0.0,0.0,0.0,16.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,3.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,2.0477,66.485,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
x_train_df.at[332, 10] = 0.269756
x_train_df.at[332, 11] = -0.462673
x_train_df.at[332, 12] = 0.462673
x_train_df.at[332, 13] = 0.269756
x_train = x_train_df.values

### Creating the Model

In [6]:
svm_params = [
    {
        "C": 0.001,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 0.01,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 0.1,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 1,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 10,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 100,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": True,
    },
    {
        "C": 0.001,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 0.01,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 0.1,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 1,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 10,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 100,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": True,
    },
    {
        "C": 0.001,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 0.01,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 0.1,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 1,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 10,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 100,
        "kernel": "rbf",
        "gamma": "scale",
        "shrinking": False,
    },
    {
        "C": 0.001,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    },
    {
        "C": 0.01,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    },
    {
        "C": 0.1,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    },
    {
        "C": 1,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    },
    {
        "C": 10,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    },
    {
        "C": 100,
        "kernel": "rbf",
        "gamma": "auto",
        "shrinking": False,
    }
]

In [7]:
print(len(svm_params))

24


In [8]:
with open("./training_testing_results.csv", "w") as log_file:
    log_file.write("c,kernel,gamma,shrinking,int_precision,int_recall,int_fbeta,ext_precision,ext_recall,ext_fbeta\n")
    for n, model_params in enumerate(svm_params):
        print(n)
        # log_file.write(f"Model {n}\n")
        # log_file.write(f"Penalty: {model_params["penalty"]}, C: {model_params["C"]}\n")
        log_file.write(f"{model_params['C']},{model_params['kernel']},{model_params['gamma']},{model_params['shrinking']},")
        
        # first we create a model with one of the optimized parameter sets
        print("model fits")
        svm_model = SVC(C=model_params["C"], kernel=model_params["kernel"], 
                        gamma=model_params["gamma"], shrinking=model_params["shrinking"], random_state=42)
        svm_model.fit(x_train, y_train)

        # then a model trained using these parameters is tested on the internal test set
        # I use the classes as predictions, because the confusion matrix is calculated like this
        # however, I think the production model should return probabilities
        model_predictions = svm_model.predict(internal_x_test)
        print("model predicted")

        # the values of the testing on the internal test set are then used to select the best model
        # ? which <average> to select?? 
        # selected micro, idk why
        precision, recall, fbeta, support = precision_recall_fscore_support(internal_y_test, model_predictions, average='micro')
        log_file.write(f"{precision},{recall},{fbeta},")
        
        print("int stats calculated")

        # then the model is tested on an external test set to assess its "real" performance
        model_predictions_for_external = svm_model.predict(external_x_test)
        ext_precision, ext_recall, ext_fbeta, ext_support = precision_recall_fscore_support(external_y_test, model_predictions_for_external, average='micro')
        print("ext stats calculated")
        log_file.write(f"{ext_precision},{ext_recall},{ext_fbeta}\n")

0
model fits
model predicted
int stats calculated
ext stats calculated
1
model fits
model predicted
int stats calculated
ext stats calculated
2
model fits
model predicted
int stats calculated
ext stats calculated
3
model fits
model predicted
int stats calculated
ext stats calculated
4
model fits
model predicted
int stats calculated
ext stats calculated
5
model fits
model predicted
int stats calculated
ext stats calculated
6
model fits
model predicted
int stats calculated
ext stats calculated
7
model fits
model predicted
int stats calculated
ext stats calculated
8
model fits
model predicted
int stats calculated
ext stats calculated
9
model fits
model predicted
int stats calculated
ext stats calculated
10
model fits
model predicted
int stats calculated
ext stats calculated
11
model fits
model predicted
int stats calculated
ext stats calculated
12
model fits
model predicted
int stats calculated
ext stats calculated
13
model fits
model predicted
int stats calculated
ext stats calculated
14

In [9]:
best_svm_model = SVC(C=10, kernel="rbf", gamma="scale", shrinking=True)

In [10]:
best_svm_model.fit(x_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(internal_y_test, best_svm_model.predict(internal_x_test))
# fpr, tpr, thresholds = roc_curve(internal_y_test, best_svm_model.predict_proba(internal_x_test)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('svm_ROC.png', dpi=600)
# plt.show()

In [12]:
logit_roc_auc

0.5151515151515151

In [13]:
with open("./2drdkit_autoencoder_366_svm.pickle", "wb") as file:
    pickle.dump(best_svm_model, file)